# Project: planning my next holidays ☀️

Let's create a script that allows to get some information about all the hotels in a given city on <a href="https://www.booking.com" target="_blank">www.booking.com</a> 🧙

**We strongly recommend that you use Scrapy, it will be much easier!**

You can scrap as many information as you want, but we suggest that you get at least:

* The hotel name, 
* The url to its booking.com page, 
* Its coordinates: latitude and longitude,
* The score given by the website users,
* The text description of the hotel.

Then, you can execute this script for several cities from yesterday's list. Make sure you save the results in different files for each city and that the name of the city is stored in the filename (for later purposes 😉).

In [1]:
!pip install scrapy

In [2]:
import json
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [3]:
destination_name = 'Annecy'

In [4]:
class Hotels(scrapy.Spider):
    # Name of your spider
    name = "hotels"

    # Starting URL
    start_urls = ['https://www.booking.com/index.fr.html']

    # Parse function for login
    def parse(self, response):
        # FormRequest used to login
        return scrapy.FormRequest.from_response(
            response,
            formdata={'ss': destination_name},
            callback=self.after_search
        )

    # Callback used after login
    def after_search(self, response):
        
        hotels = response.css('.sr_item')

        for h in hotels:
            yield {
                'name': h.css('.sr-hotel__name::text').get(),
                'url': "https://www.booking.com" + h.css('.hotel_name_link').attrib["href"],
                'coords': h.css('.sr_card_address_line a').attrib["data-coords"],
                'score': h.css('.bui-review-score__badge::text').get(),
                'description': h.css('.hotel_desc::text').get()
                
            }
        
        
        # Select the NEXT button and store it in next_page
        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)

In [5]:

filename = "2_hotels_" + destination_name.replace(" ", "-") + ".json"

if filename in os.listdir('/home/jovyan/Full stack Alexis/res/'):
        os.remove('/home/jovyan/Full stack Alexis/res/' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        '/home/jovyan/Full stack Alexis/res/' + filename: {"format": "json"},
    }
})

process.crawl(Hotels)
process.start()

2021-10-16 14:02:37 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-10-16 14:02:37 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-5.4.129+-x86_64-with-glibc2.10
2021-10-16 14:02:37 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-10-16 14:02:37 [scrapy.extensions.telnet] INFO: Telnet Password: f5767e16f315870c
2021-10-16 14:02:37 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-10-16 14:02:37 [scrapy.middleware] INFO: Enab